In [ ]:
from keras.models import load_model
model=load_model('two_strand_model.h5')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
c = 0
for i, row in df.iterrows():
    encoding_promoter[c,:row.promoter_len*2,:]=double_seq2onehot(CHROM2SEQ[row.chr][row.promoterDHSstart:row.promoterDHSend])
    encoding_distal[c,:row.distal_len*2,:]=double_seq2onehot(CHROM2SEQ[row.chr][row.distalDHSstart:row.distalDHSend])
    c += 1
    

In [ ]:
"""
Load Test Data
"""
train_ind, test_ind, y_train, y_test = train_test_split(range(len(Y)), Y, test_size=0.20, random_state=42)
X_promoter_train = encoding_promoter[train_ind,:,:]
X_promoter_test = encoding_promoter[test_ind,:,:]

X_distal_train = encoding_distal[train_ind,:,:]
X_distal_test = encoding_distal[test_ind,:,:]

In [ ]:
"""
Visualizing the distal filters
"""
filters=model.get_layer('distalDHSconv').get_weights()[0]
fig, axs = plt.subplots(4,4, figsize=(15, 6), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()

for i in range(16):
    """
    for k in range(6):
        for j in range(4):
            filters[k,j,i]=(filters[k,j,i]+min(filters[k,:,i]))/(min(filters[k,:,i])+max(filters[k,:,i]))
            """
    im=axs[i].imshow(filters[:,:,i].T)
    #plt.yticks(range(4),['A','T','C','G'])
    plt.colorbar(im,ax=axs[i])
plt.title('Filters')
plt.show()

In [ ]:
"""
ROC score
"""
#y_score = model.predict_proba(xd_dev, verbose=0)
print('Predicting on test data')
y_score = model.predict([X_distal_test,X_promoter_test])
score=roc_auc_score(y_test, y_score, average='macro', sample_weight=None)
print(score)